In [1]:
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output
import numpy as np
from ipywidgets import Video, Image
from glob import glob
import os
import time

In [2]:
def analysis(image_paths, result_path): #image array    
    def result_paths(result_path): #get paths from result.txt
        paths =[]
        with open(result_path, 'r') as f:
            size = os.path.getsize(result_path)
            if size != 0:
                for line in f:
                    paths.append(line.split(' ')[0]) 
                return paths
            else:
                paths = []
                return paths
            
    def next_item(): 
        clear_output()
        analysis(new_img_paths, result_path)

    start         = time.time()
    all_paths     = result_paths(result_path)
    
    new_img_paths = [path for path in image_paths if path['img_path'] not in all_paths]
    
#     for path in image_paths:
#         if path['img_path'] in all_paths:
#             print(path['img_path'])
            
    
    if len(new_img_paths) == 0:
        print("CONGRATS...TASK FINITO !!")
        return 0
    
    rand_seed  = np.random.choice(range(0, 100))
    np.random.seed(rand_seed)
    rand_index = np.random.choice(range(0, len(new_img_paths)))
    
    rand_path  = new_img_paths[rand_index]
    img_path   = rand_path['img_path']
    video_path = rand_path['vid_path']
    
    print(img_path, video_path)
    
    #radio buttons
    gender = widgets.RadioButtons(
        options=['good', 'bad'],
        value=None,
        layout={'width': 'max-content'}, 
        description='Gender:',
        disabled=False
    )

    color = widgets.RadioButtons(
        options=['good', 'bad'],
        value=None, 
        layout={'width': 'max-content'}, 
        description='Color :',
        disabled=False
    )

    category = widgets.RadioButtons(
        options=['good', 'bad'],
        value=None, 
        layout={'width': 'max-content'},
        description='Category:',
        disabled=False
    )

    pattern = widgets.RadioButtons(
        options=['good', 'bad'],
        value=None,
        layout={'width': 'max-content'},
        description='Pattern:',
        disabled=False
    )   
    
    frame = widgets.RadioButtons(
        options=['good', 'bad'],
        value=None,
        layout={'width': 'max-content'},
        description='Frame:',
        disabled=False
    ) 

    #submit button
    submit = widgets.Button(description = 'Submit')  
    def click_submit(arg):
        if gender.value and color.value and category.value and pattern.value is not None:
            with open(result_path, 'a') as f:
                f.write(f"{img_path} {gender.value} {color.value} {category.value} {pattern.value} {frame.value}\n")
            print("recorded")
            next_item()
        else:
            print('please fill up all section')    
            
    submit.on_click(click_submit)
    
    start = time.time()
    #read image and vide
    final_image = Image.from_file(img_path,   height = 500, width =500)
    video       = Video.from_file(video_path, width = 500, height=500, autoplay=False)  
    
    #display 
    print("--> "+ img_path)
    content = widgets.HBox([video, final_image])
    radio = widgets.HBox([gender,color, category,pattern,frame])
    display(widgets.VBox([content,radio, submit, widgets.Output()]))
    
    print(time.time() - start)

In [3]:
video_root       = '/notebooks/common-dataset/common/test_mx/video_segmented'
all_videos       = glob(video_root+'/*/*')
images_root_path = '/notebooks/common-dataset/common/test_mx/Suggested_ads'
img_paths        =  glob(images_root_path+"/*/multi_color_image_matching/*/*.png")
result_path      = '/notebooks/common-dataset/common/test_mx/result.txt'

In [4]:
ind     = 48
series  = img_paths[ind].split('/')[-3]
episode = img_paths[ind].split('/')[-1]
print(img_paths[ind])
print(len(series), episode[:len(series)+4])
video_path = f"{video_root}/{series}/{episode[:len(series)+4]}.mp4"
print(video_path)

/notebooks/common-dataset/common/test_mx/Suggested_ads/Marvels_ThePunisher/multi_color_image_matching/Marvels_ThePunisher_009_1_M_H7S6V1_half_tshirt.jpg/781da07b-98f2-43a5-97d7-c2292789cc8c.png
26 781da07b-98f2-43a5-97d7-c22927
/notebooks/common-dataset/common/test_mx/video_segmented/multi_color_image_matching/781da07b-98f2-43a5-97d7-c22927.mp4


In [5]:
new_img_vid_path = []
for img_path in img_paths:
    req_vid = None
    for vid_path in all_videos:
        if (vid_path.split('/')[-1].split('.')[0]) in img_path:     
            new_img_vid_path.append({'img_path':img_path, 'vid_path':vid_path})
            break
    
        
print(len(new_img_vid_path)) 

99


In [6]:
with open(result_path, 'a') as f:
    pass

analysis(new_img_vid_path, result_path)

/notebooks/common-dataset/common/test_mx/Suggested_ads/Scam_1992/multi_color_image_matching/Scam_1992_013_2_M_H11S1V7_full_shirt.jpg/5c39d46d-4565-44ab-b5d4-bbf6ebc35473.png /notebooks/common-dataset/common/test_mx/video_segmented/Scam_1992/Scam_1992_013.mp4
--> /notebooks/common-dataset/common/test_mx/Suggested_ads/Scam_1992/multi_color_image_matching/Scam_1992_013_2_M_H11S1V7_full_shirt.jpg/5c39d46d-4565-44ab-b5d4-bbf6ebc35473.png


0.26151490211486816
